In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
from os.path import join as pjoin
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pingouin as pg

sys.path.append("../..")
import circletrack_behavior as ctb
import circletrack_neural as ctn
import plotting_functions as pf

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [55]:
## Set parameters
behavior_path = '../../../MultiCon_Imaging/'
fig_path = '../../../MultiCon_Imaging/MultiCon_Control1/intermediate_figures/'
num_experiments = 1 + 1 ## change first value since np.arange is not inclusive
experiment_list = ['MultiCon_Control{}'.format(x) for x in np.arange(1, num_experiments)]
session_dict = {'mc13': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
                               'A11', 'A12', 'A13', 'A14', 'A15', 'B1', 'B2', 'B3', 'B4', 'B5'],
                'mc14': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
                               'A11', 'A12', 'A13', 'A14', 'A15', 'B1', 'B2', 'B3', 'B4', 'B5']}
colors = [pf.colors('mc_control', 'control')]

In [45]:
## Calculate lick accuracy, dprime metrics during non-probe
experiment_pc = {}
experiment_pc_trial = {}
experiment_sdt = {}
for experiment in experiment_list:
    bpath = pjoin(behavior_path, experiment)
    bpath = pjoin(bpath, 'output/behav')
    mouse_pc = {}
    mouse_pc_trial = {}
    mouse_sdt = {}
    for mouse in os.listdir(bpath):
        path = pjoin(bpath, mouse)
        percent_correct = {}
        percent_correct_trials = {}
        sdt_metrics = {}
        for session in np.arange(0, len(session_dict[mouse])):
            behav = pd.read_feather(pjoin(path, '{}_{}.feat'.format(mouse, session_dict[mouse][session])))
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            behav_nonprobe = behav.loc[~behav['probe']]
            forward_trials, reverse_trials = ctb.get_forward_reverse_trials(behav)
            forward_df = behav.loc[(behav['trials'].isin(forward_trials))]
            forward_df = forward_df.loc[~forward_df['probe']]
            ## Percent correct licking
            pc = ctb.lick_accuracy(behav_nonprobe, port_one=reward_one, port_two=reward_two, by_trials=False)
            pc_trials = ctb.lick_accuracy(forward_df, port_one=reward_one, port_two=reward_two, by_trials=True)
            percent_correct[session] = pc
            percent_correct_trials[session] = pc_trials
            ## dprime, correct rejection, hit rate
            metrics = ctb.dprime_metrics(forward_df, reward_one, reward_two, reward_index='one', forward_reverse='forward')
            sdt_metrics[session] = metrics
        mouse_pc[mouse] = percent_correct
        mouse_pc_trial[mouse] = percent_correct_trials
        mouse_sdt[mouse] = sdt_metrics
    experiment_pc[experiment] = mouse_pc
    experiment_pc_trial[experiment] = mouse_pc_trial
    experiment_sdt[experiment] = mouse_sdt

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Control/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Control/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Control/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Control/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Control/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTr

In [67]:
## Percent correct data non_probe
accuracy = pd.DataFrame()
for experiment in experiment_pc:
    for mouse in experiment_pc[experiment]:
        if mouse == 'mc_EEG2_01':
            next
        else:
            df = pd.DataFrame(experiment_pc[experiment][mouse], index=[0]).T.rename(columns={0: 'percent_correct'})
            df.insert(0, 'mouse', mouse)
            df.insert(1, 'day', [x for x in np.arange(1, 21)])
            accuracy = pd.concat([accuracy, df], axis=0, ignore_index=True)

fig = pf.plot_behavior_across_days(accuracy, x_var='day', y_var='percent_correct', groupby_var=['day'], titles=['Lick Accuracy'], expert_line=False,
                                   x_title='Day', y_title='Percent Correct (%)', template='simple_white', avg_color=colors[0], plot_transitions=[15.5])
fig.update_xaxes(dtick=2) 
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy.png'))

In [36]:
## Get performance drop for each mouse
performance_drop = ctb.performance_drop(accuracy, day_list=[5, 10, 15], replace=True)
performance_drop = performance_drop.replace({5: 'A5 to A6', 10: 'A10 to A11', 15: 'A15 to B1'})
avg_drop = performance_drop.groupby(['day'], as_index=False).agg({'drop': ['mean', 'sem']})
avg_drop_df = avg_drop['drop']
avg_drop_df.insert(0, 'day', avg_drop['day'])
avg_drop_df = avg_drop_df.reindex([2, 0, 1]).reset_index(drop=True)
## Plot fig
fig = pf.custom_graph_template(x_title='', y_title='Performance Difference (%)')
fig.add_trace(go.Bar(x=avg_drop_df['day'], y=avg_drop_df['mean'],
                     error_y=dict(type = 'data', array=avg_drop_df['sem'], thickness = 2.5, width = 10),
                     marker_color='darkgrey', marker_line_color='black',
                     marker_line_width=2, showlegend=False, width=[0.5, 0.5, 0.5]))
fig.add_trace(go.Scatter(x=performance_drop['day'], y=performance_drop['drop'], mode='markers', 
                            marker_color='black', showlegend=False))
fig.show()
fig.write_image(pjoin(fig_path, 'performance_drop.png'))

In [38]:
pg.homoscedasticity(data=performance_drop, dv='drop', group='day')

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/scipy/stats/_morestats.py:2643: RuntimeWarning:

divide by zero encountered in double_scalars



,W,pval,equal_var
levene,inf,0.0,False


In [39]:
performance_drop.anova(dv='drop', between='day', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,day,444.622304,2,222.311152,1.351705,0.381487,0.473999
1,Within,493.401693,3,164.467231,NaN,NaN,NaN


In [40]:
## Calculate percent correct during probe
experiment_pc_probe = {}
for experiment in experiment_list:
    bpath = pjoin(behavior_path, experiment)
    bpath = pjoin(bpath, 'output/behav')
    mouse_pc = {}
    for mouse in os.listdir(bpath):
        path = pjoin(bpath, mouse)
        percent_correct = {}
        for session in np.arange(0, len(session_dict[mouse])):
            behav = pd.read_feather(pjoin(path, '{}_{}.feat'.format(mouse, session_dict[mouse][session])))
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            probe_behav = behav.loc[behav['probe']]
            pc = ctb.probe_lick_accuracy(probe_behav, port_one=reward_one, port_two=reward_two)
            percent_correct[session] = pc
        mouse_pc[mouse] = percent_correct
    experiment_pc_probe[experiment] = mouse_pc

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Control/analysis/../../circletrack_behavior.py:978: RuntimeWarning:

invalid value encountered in long_scalars



In [66]:
## Percent correct probe
accuracy_probe = pd.DataFrame()
for experiment in experiment_pc_probe:
    for mouse in experiment_pc_probe[experiment]:
        if mouse == 'mc_EEG2_01':
            next
        else:
            df = pd.DataFrame(experiment_pc_probe[experiment][mouse], index=[0]).T.rename(columns={0: 'percent_correct'})
            df.insert(0, 'mouse', mouse)
            df.insert(1, 'day', [x for x in np.arange(1, 21)])
            accuracy_probe = pd.concat([accuracy_probe, df], axis=0, ignore_index=True)
accuracy_probe = accuracy_probe[accuracy_probe['day'] > 1]
fig = pf.plot_behavior_across_days(accuracy_probe, x_var='day', y_var='percent_correct', groupby_var=['day'], titles=['Lick Accuracy During Probe'], expert_line=False,
                                   x_title='Day', y_title='Percent Correct (%)', template='simple_white', avg_color=colors[0], plot_transitions=[15.5])
fig.update_xaxes(dtick=2)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_accuracy.png'))

In [48]:
## d' across days
dprime_df = pd.DataFrame()
for experiment in experiment_pc:
    for mouse in experiment_pc[experiment]:
        if mouse == 'mc_EEG2_01':
            next
        else:
            df = pd.DataFrame(experiment_sdt[experiment][mouse]).T
            d = []
            for idx in np.arange(0, len(df['dprime'])):
                d.append(np.nanmean(df['dprime'][idx]))
            dprime = pd.DataFrame(d).rename(columns={0: 'dprime'})
            dprime.insert(0, 'mouse', mouse)
            dprime.insert(1, 'day', [x for x in np.arange(1, 21)])
            dprime_df = pd.concat([dprime_df, dprime], axis=0, ignore_index=True)

fig = pf.plot_behavior_across_days(dprime_df, x_var='day', y_var='dprime', groupby_var=['day'], titles=[''], plot_transitions=[15.5],
                                   x_title='Day', y_title="d'", template='simple_white', avg_color=colors[0])
fig.show()

In [51]:
performance_drop

,mouse,drop,day
0,mc13,0.000000,A5 to A6
1,mc14,12.439143,A5 to A6
0,mc13,0.000000,A10 to A11
1,mc14,7.894737,A10 to A11
0,mc13,37.110341,A15 to B1
1,mc14,9.366077,A15 to B1


In [59]:
## Plot for performance drop vs percent overlap
## THIS IS FAKE OVERLAP DATA
performance_dict = {'mc_13': {'drop': [0, 0, 37], 'overlap': [80, 69, 29]},
                    'mc_14': {'drop': [12.44, 7.89, 9.37], 'overlap': [76, 59, 34]}}
df = pd.DataFrame(performance_dict)
y_data = np.concatenate((df.loc['drop', 'mc_13'], df.loc['drop', 'mc_14']))
x_data = np.concatenate((df.loc['overlap', 'mc_13'], df.loc['overlap', 'mc_14']))
lm = pg.linear_regression(X=x_data, y=y_data, as_dataframe=False)
lm_x = []
for n in np.arange(len(lm['X'])):
    lm_x.append(lm['X'][n][1])
mouse_colors = ['darkgrey', colors[0]]
fig = pf.custom_graph_template(x_title='Neuronal Overlap (%)', y_title='Performance Drop (%)')
for idx, mouse in enumerate(df.columns):    
    fig.add_trace(go.Scatter(x=df.loc['overlap', mouse], y=df.loc['drop', mouse], 
                             mode='markers', marker=dict(color=mouse_colors[idx]), name=mouse))
fig.add_trace(go.Scatter(x=lm_x, y=lm['pred'], mode='lines', line_color=colors[0], showlegend=False))
fig.show()
fig.write_image(pjoin(fig_path, 'perf_drop_vs_overlap.png'))